In [1]:
import pickle, random, copy, platform, os, time
import numpy as np
import tensorflow as tf
import email_sending
import tflearn
from PIL import Image, ImageDraw
from __future__ import division, print_function, absolute_import

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
def read_data():
    with open('resnet_data', 'rb') as f:
        data = pickle.load(f)
        X = data[0]
        X = np.reshape(X, (-1,75,75,2))
        Y = data[1]
    return X, Y

# expresses the label data in one-hot encoding.
def onehot_encoding (Y, y_class):
    Y_onehot = np.zeros((Y.size, int(y_class)))
    for i in range(Y.size):
        Y_onehot[i][Y[i]] = 1
    return Y_onehot

In [3]:
def resnet32(x, classes, n = 5): #classes =2
    net = tflearn.conv_2d(x, 16, 3, regularizer='L2', weight_decay=0.0001)
    net = tflearn.residual_block(net, n, 16)
    net = tflearn.residual_block(net, 1, 32, downsample=True)
    net = tflearn.residual_block(net, n - 1, 32)
    net = tflearn.residual_block(net, 1, 64, downsample=True)
    net = tflearn.residual_block(net, n - 1, 64)
    net = tflearn.batch_normalization(net)
    net = tflearn.activation(net, 'relu')
    net = tflearn.global_avg_pool(net)
    net = tflearn.fully_connected(net, classes, activation='softmax')

    return net

In [4]:
def train(num_epochs=10):
    batch_size = 100
    classes = 2

    img_prep = tflearn.ImagePreprocessing()
    img_prep.add_featurewise_zero_center()
    img_prep.add_featurewise_stdnorm()

    img_aug = tflearn.ImageAugmentation()
    img_aug.add_random_flip_leftright()
    #img_aug.add_random_rotation(max_angle=25)
    img_aug.add_random_crop([75,75,2], padding=4)

    Input = tflearn.input_data(shape=[None, 75, 75, 2], dtype='float', data_preprocessing=img_prep,data_augmentation=img_aug)


    totalX, totalY = read_data()
    totalX, totalY = tflearn.data_utils.shuffle(totalX, totalY)
    totalY=onehot_encoding(totalY, classes)
    X, Y = totalX[:1200], totalY[:1200]
    testX, testY =totalX[1200:1600], totalY[1200:1600]

    network = tflearn.regression(resnet32(Input,classes),optimizer='adam',loss='categorical_crossentropy')
    
    model = tflearn.DNN(network,tensorboard_verbose=3,checkpoint_path='resnet32')
    model.fit(X, Y, n_epoch=num_epochs, shuffle=True, validation_set=(testX, testY),
              show_metric=True, batch_size=batch_size, run_id='resnet_iceberg')

if __name__ == '__main__':
    train(60)

Training Step: 719  | total loss: 0.17729 | time: 106.093s
| Adam | epoch: 060 | loss: 0.17729 - acc: 0.9491 -- iter: 1100/1200
Training Step: 720  | total loss: 0.17154 | time: 119.992s
| Adam | epoch: 060 | loss: 0.17154 - acc: 0.9502 | val_loss: 0.55325 - val_acc: 0.8050 -- iter: 1200/1200
--
INFO:tensorflow:C:\Users\User\Desktop\kaggle\resnet32-720 is not in all_model_checkpoint_paths. Manually adding it.
